## 05_finetune_transformers.ipynb — Дообучение языковых моделей на собственных данных

В этом ноутбуке реализовано дообучение (fine-tuning) современных языковых моделей (Roberta, MiniLM) на собственных данных по вакансиям hh.ru.

### Для чего нужен этот файл?

- Здесь происходит адаптация предобученных трансформеров (Roberta, MiniLM) к конкретным задачам HR-аналитики:
    - **извлечение навыков (Skill-NER)** из описания вакансий,
    - **классификация грейда (junior/middle/senior/lead)** по тексту вакансии.
- В ноутбуке описан процесс подготовки данных, запуска обучения, сохранения моделей и анализа качества.
- Результаты этого этапа лягут в основу интеллектуальных функций Telegram-бота и позволят применять ML к реальным задачам рынка труда.

**Задачи ноутбука:**
- Подготовить и загрузить датасеты для дообучения моделей.
- Запустить fine-tuning Roberta для задачи извлечения навыков.
- Запустить fine-tuning MiniLM (или Roberta) для задачи классификации грейда.
- Оценить качество моделей на отложенной выборке.
- Сохранить обученные модели для дальнейшей интеграции в проект.


In [20]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [21]:
# Загружаем обучающую выборку для дообучения моделей Skill-NER (Roberta) и классификатора грейда
# ner_df — содержит пары (описание вакансии, список навыков), используется для обучения Skill-NER
# grade_df — содержит описание/название и метку класса грейда, для обучения классификатора грейда

import pandas as pd
import json

# Загружаем датасет для Skill-NER (jsonlines с парами "description", "skills_list")
with open('data/raw/ner_train.json', 'r', encoding='utf-8') as f:
    lines = [json.loads(line) for line in f]
ner_df = pd.DataFrame(lines)

# Просматриваем первые строки и размер выборки
display(ner_df.head(), ner_df.shape)

# Загружаем датасет для классификатора грейда (csv с колонками "description", "title", "grade_class")
grade_df = pd.read_csv('data/raw/grade_train.csv')
display(grade_df.head(), grade_df.shape)

,description,skills_list
0,<p>Вакансия &quot;Водитель с личным автомобиле...,[вождение автомобилей представительского класс...
1,<p><strong>О компании и команде</strong></p> <...,"[javascript, react, css, node.js, typescript, ..."
2,<p>​​​​​​Направление студенческих медицинских ...,"[ответственность, стрессоустойчивость]"
3,<div> <p><strong>edna – аккредитованная IT-ком...,"[react, html, css, redux, typescript, webpack]"
4,<p><strong>CAD Exchanger</strong> – IT-компани...,"[git, javascript, 3d моделирование, ооп, webgl..."


(1709, 2)

,description,title,grade_class
0,<p>Вакансия &quot;Водитель с личным автомобиле...,Водитель с личным автомобилем,3
1,<p><strong>О компании и команде</strong></p> <...,Middle/Senior Frontend разработчик,2
2,"<p><strong><em>Крупная, стабильно развивающаяс...",Упаковщик,0
3,<p><strong>Приглашаем Управляющего семейным ре...,Управляющий рестораном семейного концепта,1
4,<p>​​​​​​Направление студенческих медицинских ...,Медицинский работник (Российские студенческие ...,0


(2995, 3)

### Подготовка данных для обучения модели извлечения навыков (Skill-NER)

На этом этапе мы формируем обучающую выборку для задачи многоклассовой классификации (multilabel classification) навыков по тексту вакансии:

- Из всего корпуса вакансий собирается список уникальных навыков.
- Для каждой строки создаётся бинарный вектор признаков (“1”, если навык присутствует в описании; “0”, если нет).
- Таким образом, каждая обучающая пара — это текст описания и массив меток, соответствующих наличию навыков.
- Такой формат идеально подходит для дообучения современных трансформеров (Roberta, MiniLM) на задачу multilabel classification.

Этот шаг позволяет подготовить данные к обучению модели, способной автоматически определять набор требуемых навыков по любому тексту вакансии.


In [22]:
# Формируем список всех уникальных навыков по датасету
all_skills = set()
for skills in ner_df['skills_list']:
    all_skills.update(skills)
all_skills = sorted(list(all_skills))
print("Уникальных навыков:", len(all_skills))
print("Примеры навыков:", all_skills[:10])

Уникальных навыков: 2428
Примеры навыков: [');', '.net core', '.net framework', '1c', '1c erp', '1c: erp', '1c: бухгалтерия', '1c: зарплата и кадры', '1c: предприятие', '1водительское удостоверение категории b']


In [23]:
# Создаём бинарный вектор для каждого примера (1 — навык присутствует)
import numpy as np

def skills_to_vector(skills):
    return [int(skill in skills) for skill in all_skills]

ner_df['target_vector'] = ner_df['skills_list'].apply(skills_to_vector)
print("Пример target_vector:", ner_df['target_vector'].iloc[0])
# Смотрим пример: текст, список навыков, бинарный вектор
display(ner_df[['description', 'skills_list', 'target_vector']].head())

Пример target_vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,description,skills_list,target_vector
0,<p>Вакансия &quot;Водитель с личным автомобиле...,[вождение автомобилей представительского класс...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,<p><strong>О компании и команде</strong></p> <...,"[javascript, react, css, node.js, typescript, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,<p>​​​​​​Направление студенческих медицинских ...,"[ответственность, стрессоустойчивость]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,<div> <p><strong>edna – аккредитованная IT-ком...,"[react, html, css, redux, typescript, webpack]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,<p><strong>CAD Exchanger</strong> – IT-компани...,"[git, javascript, 3d моделирование, ооп, webgl...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Подготовка данных для классификации грейда вакансии

На этом этапе мы анализируем и балансируем обучающий датасет для задачи классификации грейда (junior/middle/senior/lead):
- Смотрим распределение по классам, чтобы выявить возможный дисбаланс в данных.
- Проверяем примеры для каждого класса — это поможет понять специфику описаний вакансий и качество разметки.
- Этот анализ важен для успешного дообучения трансформера и повышения точности предсказания грейда вакансии.


In [24]:
# Показываем, сколько примеров в каждом классе (0=junior, 1=middle, 2=senior, 3=lead)
print(grade_df['grade_class'].value_counts())

grade_class
1    1315
0     774
2     688
3     218
Name: count, dtype: int64


In [25]:
# Посмотрим по 2 примера для каждого грейда
for grade in sorted(grade_df['grade_class'].unique()):
    print(f"\n==== Грейд: {grade} ====")
    print(grade_df[grade_df['grade_class'] == grade][['title', 'description']].head(2))


==== Грейд: 0 ====
                                               title  \
2                                          Упаковщик   
4  Медицинский работник (Российские студенческие ...   

                                         description  
2  <p><strong><em>Крупная, стабильно развивающаяс...  
4  <p>​​​​​​Направление студенческих медицинских ...  

==== Грейд: 1 ====
                                       title  \
3  Управляющий рестораном семейного концепта   
6                     Junior Web-разработчик   

                                         description  
3  <p><strong>Приглашаем Управляющего семейным ре...  
6  <p><strong>CAD Exchanger</strong> – IT-компани...  

==== Грейд: 2 ====
                                title  \
1  Middle/Senior Frontend разработчик   
5                Frontend-разработчик   

                                         description  
1  <p><strong>О компании и команде</strong></p> <...  
5  <div> <p><strong>edna – аккредитованная IT-ком...  

==== Г

### Формирование обучающей и тестовой выборки для классификатора грейда

- Для повышения качества классификации объединяем название и описание вакансии в единый текстовый признак.
- Разбиваем выборку на обучающую (80%) и тестовую (20%) части, сохраняя пропорции классов (stratify).
- Такой подход позволяет учесть максимум информации о вакансии и обеспечивает объективную оценку качества модели на независимых данных.


In [26]:
# Соединим title и description для лучшего контекста
grade_df['text'] = grade_df['title'].fillna('') + ' ' + grade_df['description'].fillna('')

In [27]:
from sklearn.model_selection import train_test_split

X = grade_df['text'].values
y = grade_df['grade_class'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train size:", len(X_train))
print("Test size:", len(X_test))

Train size: 2396
Test size: 599


### Baseline-модель для классификации грейда вакансии

Для быстрой проверки пригодности данных и минимальной оценки качества используем простую ML-модель: векторизацию текста TF-IDF и классификатор LogisticRegression.
- Такая модель даёт baseline-результат, с которым можно сравнивать более сложные подходы (Roberta, MiniLM).
- Метрики (accuracy, precision, recall, f1) покажут, насколько текст вакансии отражает уровень позиции.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Обучение логистической регрессии
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_tfidf, y_train)

# Предсказание на тестовой выборке
y_pred = clf.predict(X_test_tfidf)

# Оценка качества
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Классификационный отчёт:\n", classification_report(y_test, y_pred, digits=3))

Accuracy: 0.5742904841402338
Классификационный отчёт:
               precision    recall  f1-score   support

           0      0.738     0.490     0.589       155
           1      0.558     0.837     0.670       263
           2      0.465     0.341     0.393       138
           3      1.000     0.023     0.045        43

    accuracy                          0.574       599
   macro avg      0.690     0.423     0.424       599
weighted avg      0.615     0.574     0.540       599



#### Выводы по baseline-модели классификации грейда (TF-IDF + LogisticRegression)

- **Accuracy (доля правильных ответов):** 0.57 — базовая модель верно классифицирует около 57% вакансий по грейду.
- **Лучше всего определяется класс "middle" (recall 0.84)** — модель часто относит вакансии к нему (но precision ниже 0.56, возможен перекос).
- **Junior определяется достаточно уверенно** (precision 0.74, f1-score 0.59), но recall ниже (0.49) — то есть не все junior-вакансии ловит.
- **Senior/lead определяются хуже**:
    - Для senior (2): f1-score 0.39 — низкий recall (0.34), часть таких вакансий путается с другими классами.
    - Для lead (3): очень высокая precision (1.00), но recall только 0.02 (модель почти не предсказывает этот класс, f1-score всего 0.04).
- **Макро-усреднённый f1-score:** 0.42 — базовый ориентир для улучшения.

##### Причины и интерпретация:
- **Перекос классов:** больше всего примеров middle/junior, меньше lead — модель склонна "игнорировать" редкие классы.
- **TF-IDF не учитывает контекст и синонимы, не работает с “тонкими” различиями между senior/lead.**
- Это ожидаемо для baseline — сложные модели (трансформеры) должны показать прирост качества, особенно по малым и “тонким” классам.

##### Что делать дальше:
- Применить Roberta/MiniLM — они должны лучше различать смыслы и повысить recall по senior/lead.
- При необходимости — балансировать классы, использовать веса/аугментацию, дообучить модель на конкретных примерах.

**Базовая модель даёт рабочую отправную точку. Результат показывает, что текст вакансии частично отражает грейд, но есть большой потенциал для улучшения с помощью современных методов.**


Baseline-модель (TF-IDF + LogisticRegression) даёт accuracy 57%.
Лучше всего различает middle/junior, хуже — senior/lead.
Это ожидаемо для простой модели, и подтверждает необходимость использовать современные трансформеры для повышения качества.

#### Классификация грейда вакансии с помощью sentence-transformers MiniLM

Используем современную мультиязычную модель MiniLM для обучения классификатора уровня вакансии (junior/middle/senior/lead) по тексту.  
Такая модель учитывает смысл и контекст, что позволяет повысить качество предсказаний по сравнению с простыми ML-подходами.


In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 850.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

#### Обучение и тестирование кастомного PyTorch-классификатора поверх sentence-transformers MiniLM

Модель получает sentence-эмбеддинги для текста вакансии и обучает поверх них простой линейный классификатор для определения грейда.  
Это "нейросетевой" способ, полностью на PyTorch, приближённый к профессиональным пайплайнам.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import torch.nn as nn

# Загрузка базовой модели
base_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Freeze base (по желанию)
for param in base_model.parameters():
    param.requires_grad = False

# Добавляем классификационный слой
class MiniLMClassifier(nn.Module):
    def __init__(self, embed_dim, num_classes):
        super().__init__()
        self.base = base_model
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, input_texts):
        with torch.no_grad():
            embeddings = self.base.encode(input_texts, convert_to_tensor=True)
        logits = self.classifier(embeddings)
        return logits

# Проверяем размерность эмбеддинга
embed_dim = base_model.get_sentence_embedding_dimension()
num_classes = 4
model = MiniLMClassifier(embed_dim, num_classes)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Создаём кастомный датасет
class TextDataset(Dataset):
    def __init__(self, texts, labels, base_model):
        self.texts = texts
        self.labels = labels
        self.base_model = base_model

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Получаем sentence-эмбеддинг (без .detach(), потому что будем обучать classifier)
        with torch.no_grad():
            embedding = torch.tensor(self.base_model.encode(text, convert_to_numpy=True))
        return embedding, label

# train и test датасеты
train_dataset = TextDataset(X_train, y_train, base_model)
test_dataset = TextDataset(X_test, y_test, base_model)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniLMClassifier(embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
n_epochs = 5

for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        embeddings, labels = batch
        embeddings, labels = embeddings.to(device), labels.to(device)
        outputs = model.classifier(embeddings)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * embeddings.size(0)
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs} - train loss: {avg_loss:.4f}")


Epoch 1/5 - train loss: 1.2506
Epoch 2/5 - train loss: 1.1851
Epoch 3/5 - train loss: 1.1536
Epoch 4/5 - train loss: 1.1320
Epoch 5/5 - train loss: 1.1132


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

model.eval()  # Переводим модель в режим оценки (отключает dropout/batchnorm)
all_preds = []
all_labels = []

# Отключаем градиенты для ускорения и экономии памяти
with torch.no_grad():
    for batch in test_loader:
        embeddings, labels = batch
        embeddings = embeddings.to(device)
        # Получаем логиты классификатора
        outputs = model.classifier(embeddings)
        # Выбираем класс с максимальным значением (argmax)
        preds = outputs.argmax(dim=1).cpu().numpy()
        # Собираем предсказания и реальные значения
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# Считаем accuracy и полный классификационный отчёт (precision, recall, f1 по классам)
print("MiniLMClassifier test accuracy:", accuracy_score(all_labels, all_preds))
print("Классификационный отчёт:\n", classification_report(all_labels, all_preds, digits=3))

MiniLMClassifier test accuracy: 0.4691151919866444
Классификационный отчёт:
               precision    recall  f1-score   support

           0      0.446     0.239     0.311       155
           1      0.484     0.837     0.613       263
           2      0.393     0.174     0.241       138
           3      0.000     0.000     0.000        43

    accuracy                          0.469       599
   macro avg      0.331     0.312     0.291       599
weighted avg      0.418     0.469     0.405       599



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **Выводы по MiniLMClassifier для классификации грейда**

* **Train loss** последовательно снижался (с 1.25 до 1.11 за 5 эпох) — модель училась, но обучение оказалось недостаточным для уверенной классификации всех классов.
* **Accuracy на тесте:** **0.47** — это ниже, чем у baseline TF-IDF + LogisticRegression (**0.57**).
* **Middle (1)** определяется лучше всего (recall 0.84, f1-score 0.61), но остальные классы распознаются слабо.
* **Senior/Lead:**

  * senior (2): f1-score 0.24 (низкая точность и полнота)
  * lead (3): f1-score 0.00 (модель вообще не предсказывает этот класс)
* **Macro F1-score:** **0.29** — модель практически "игнорирует" редкие классы (особенно lead).

### **Причины и интерпретация:**

* **Sentence-transformers MiniLM** в такой схеме хуже справляется с мультиклассовой задачей, если тренируется только "верхушка" (линейный слой), особенно при сильном классовом дисбалансе.
* Возможно, нужно больше эпох, балансировка классов, увеличение обучающей выборки, либо дообучение всей модели (unfreeze base\_model).
* В этом конкретном случае baseline (TF-IDF + LogisticRegression) справился лучше, чем быстрый MiniLM-классификатор.

### **Что можно сделать:**

* Если есть возможность — попробуй **разморозить base\_model и дообучать всю MiniLM** (но это дольше и требует GPU).
* Можно увеличить количество эпох (до 10–15), попробовать class weights, oversampling, аугментацию.
* Для лучшего качества на такой задаче обычно используют Roberta или специализированные модели.

---

### **Краткий вывод для отчёта/презентации:**

> **Быстрое дообучение MiniLM поверх sentence-эмбеддингов не дало выигрыша по качеству — accuracy 0.47, модель определяет в основном middle, но игнорирует сложные классы.
> Базовая ML-модель TF-IDF + LogisticRegression показала более устойчивый результат.
> Для улучшения качества требуется более глубокое дообучение трансформеров и/или балансировка классов.**



#### Дообучение Roberta на задаче классификации грейда

Используем ruRoberta (или Roberta-base), дообучаем на корпусе вакансий для задачи классификации грейда (junior/middle/senior/lead).  
Этот подход позволяет учесть глубокие семантические связи и контекст, что должно дать прирост качества по сравнению с простыми ML-моделями и MiniLM.


In [ ]:
#!pip install -U transformers datasets torch tqdm
#!pip install --upgrade --force-reinstall transformers==4.38.2 datasets==2.17.0 torch==2.2.2 tqdm --no-cache-dir
#!pip install --upgrade --force-reinstall "numpy<2" torch==2.2.2 transformers==4.38.2 datasets==2.17.0 --no-cache-dir
# Uninstall existing conflicting packages
!pip uninstall -y torch torchvision transformers datasets accelerate torchao peft
# Install compatible versions, specifying versions for transformers and peft
!pip install --upgrade --force-reinstall "numpy<2" torch==2.2.1 torchvision==0.17.1 transformers==4.38.2 datasets==2.17.0 accelerate peft==0.9.0 --no-cache-dir


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
Found existing installation: torchao 0.10.0
Uninstalling torchao-0.10.0:
  Successfully uninstalled torchao-0.10.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5

In [ ]:
import torch
print(torch.__version__)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

2.2.1+cu121


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
#Подготовка данных
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Используем ruRoberta (русская Roberta, если большинство вакансий на русском)
model_name = "ai-forever/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Используем X_train, y_train, X_test, y_test из прошлых шагов
# Преобразуем данные в формат HuggingFace Datasets
train_ds = Dataset.from_dict({
    "text": list(X_train),
    "label": list(y_train)
})
test_ds = Dataset.from_dict({
    "text": list(X_test),
    "label": list(y_test)
})

# Токенизация
def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(preprocess, batched=True)
test_ds = test_ds.map(preprocess, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2396 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

In [ ]:
#загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
#параметры обучения
training_args = TrainingArguments(
    output_dir="./roberta_grade_classifier",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,      # Можно увеличить до 5-8 для лучшего качества
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=False,
    report_to=[]   # <--- ВОТ ЭТА СТРОКА отключает wandb и все другие логгеры
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [ ]:
!pip install --upgrade --force-reinstall accelerate==0.27.2 huggingface_hub==0.23.1 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 147.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/401.3 kB 193.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 174.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 181.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 121.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 220.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 210.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 212.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 206.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.271000,1.242529,0.439065
2,1.201200,1.247520,0.439065
3,1.290900,1.243441,0.439065


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=900, training_loss=1.2668849605984158, metrics={'train_runtime': 36447.3687, 'train_samples_per_second': 0.197, 'train_steps_per_second': 0.025, 'total_flos': 1674691976687616.0, 'train_loss': 1.2668849605984158, 'epoch': 3.0})

In [ ]:
# Получить предсказания
preds = trainer.predict(test_ds)
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

print("Roberta accuracy:", accuracy_score(y_true, y_pred))
print("Roberta classification report:\n", classification_report(y_true, y_pred, digits=3))

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Roberta accuracy: 0.43906510851419034
Roberta classification report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000       155
           1      0.439     1.000     0.610       263
           2      0.000     0.000     0.000       138
           3      0.000     0.000     0.000        43

    accuracy                          0.439       599
   macro avg      0.110     0.250     0.153       599
weighted avg      0.193     0.439     0.268       599



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Цель дообучения модели MiniLM

Наша цель — получить модель, которая сможет автоматически и точно определять грейд (уровень) вакансии по её тексту на основе специфики нашего реального датасета.  
Мы выбрали MiniLM, потому что это современная и быстрая мультиязычная модель, которая хорошо работает с небольшими и средними объёмами данных и способна “понимать” смысл текстов даже при ограниченном количестве обучающих примеров.  
Дообучение позволяет адаптировать MiniLM под особенности наших вакансий и добиться максимального качества именно на наших данных.


In [ ]:
from sentence_transformers import SentenceTransformer

# Загружаем MiniLM (универсальная, быстрая, поддерживает русский)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Получаем эмбеддинги для обучающей и тестовой выборок
X_train_emb = model.encode(list(X_train), convert_to_numpy=True, show_progress_bar=True)
X_test_emb = model.encode(list(X_test), convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/75 [00:00<?, ?it/s]

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

In [34]:
from sklearn.ensemble import RandomForestClassifier

# Обучаем случайный лес с балансировкой классов
clf = RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=42)
clf.fit(X_train_emb, y_train)
y_pred = clf.predict(X_test_emb)

In [35]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

Accuracy: 0.49081803005008345
              precision    recall  f1-score   support

           0      0.705     0.200     0.312       155
           1      0.474     0.947     0.632       263
           2      0.478     0.080     0.137       138
           3      0.429     0.070     0.120        43

    accuracy                          0.491       599
   macro avg      0.521     0.324     0.300       599
weighted avg      0.532     0.491     0.398       599



Следующий шаг — повысим качество через oversampling
Теперь попробуем балансировать тренировочную выборку, чтобы модель видела больше примеров “редких” классов.
Это быстро реализуется через RandomOverSampler из imbalanced-learn.

In [ ]:
!pip install imbalanced-learn

In [47]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train_emb, y_train)

In [48]:
clf = RandomForestClassifier(n_estimators=500, class_weight='balanced', random_state=42)
clf.fit(X_train_res, y_train_res)
y_pred = clf.predict(X_test_emb)

In [49]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

Accuracy: 0.5058430717863105
              precision    recall  f1-score   support

           0      0.631     0.342     0.444       155
           1      0.495     0.837     0.622       263
           2      0.433     0.188     0.263       138
           3      0.364     0.093     0.148        43

    accuracy                          0.506       599
   macro avg      0.481     0.365     0.369       599
weighted avg      0.507     0.506     0.459       599



In [50]:
import joblib

# Сохраняем модель
joblib.dump(clf, "grade_rf_model.joblib")

# (опционально) Сохрани encodings для теста:
import numpy as np
np.save("X_test_emb.npy", X_test_emb)
np.save("y_test.npy", y_test)

In [38]:
import lightgbm as lgb

clf = lgb.LGBMClassifier(class_weight='balanced', n_estimators=200, random_state=42)
clf.fit(X_train_res, y_train_res)
y_pred = clf.predict(X_test_emb)
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.026059 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97918
[LightGBM] [Info] Number of data points in the train set: 4208, number of used features: 384
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LightGBM Accuracy: 0.5008347245409015
              precision    recall  f1-score   support

           0      0.496     0.387     0.435       155
           1      0.513     0.730     0.603       263
           2      0.483     0.312     0.379       138
           3      0.333     0.116     0.172        43

    accuracy                          0.501       599
   macro avg      0.456     0.386     0.397       599
w

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [39]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.4 MB/s eta 0:00:00


In [41]:
from catboost import CatBoostClassifier

# Создаём и обучаем CatBoost (используем class_weights="Balanced")
clf = CatBoostClassifier(
    iterations=500,
    depth=8,
    verbose=100,         # Выведет ход обучения, можно убрать или поставить 50/200
    random_state=42
)
clf.fit(X_train_res, y_train_res)
y_pred = clf.predict(X_test_emb)

Learning rate set to 0.15294
0:	learn: 1.3276686	total: 1.16s	remaining: 9m 36s
100:	learn: 0.2782559	total: 2m 5s	remaining: 8m 15s
200:	learn: 0.1341999	total: 4m 6s	remaining: 6m 7s
300:	learn: 0.0811605	total: 6m 10s	remaining: 4m 4s
400:	learn: 0.0566351	total: 8m 11s	remaining: 2m 1s
499:	learn: 0.0430778	total: 10m 10s	remaining: 0us


In [42]:
from sklearn.metrics import accuracy_score, classification_report

print("CatBoost Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

CatBoost Accuracy: 0.48747913188647746
              precision    recall  f1-score   support

           0      0.500     0.348     0.411       155
           1      0.503     0.722     0.593       263
           2      0.467     0.312     0.374       138
           3      0.238     0.116     0.156        43

    accuracy                          0.487       599
   macro avg      0.427     0.375     0.383       599
weighted avg      0.475     0.487     0.464       599



In [43]:
!pip install imbalanced-learn

In [44]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train_emb, y_train)

In [45]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(
    n_estimators=400,        # Больше деревьев — чуть выше качество
    class_weight='balanced',
    max_depth=12,            # Можно увеличивать до 16
    random_state=42
)
clf.fit(X_train_sm, y_train_sm)
y_pred = clf.predict(X_test_emb)

In [46]:
from sklearn.metrics import accuracy_score, classification_report

print("SMOTE + RF Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=3))

SMOTE + RF Accuracy: 0.48080133555926546
              precision    recall  f1-score   support

           0      0.496     0.452     0.473       155
           1      0.527     0.627     0.573       263
           2      0.400     0.290     0.336       138
           3      0.289     0.302     0.295        43

    accuracy                          0.481       599
   macro avg      0.428     0.418     0.419       599
weighted avg      0.473     0.481     0.473       599



### Выводы по экспериментам с классификацией грейда вакансий

В ходе проекта были протестированы как базовые, так и продвинутые ML-модели для автоматической классификации грейда вакансий (junior, middle, senior, lead) на реальных данных.  
Рассматривались варианты: TF-IDF + LogisticRegression, MiniLM + RandomForest, LightGBM, CatBoost, дообучение Roberta.

Наиболее стабильные результаты показали:
- TF-IDF + LogisticRegression (accuracy ~0.57, f1 до 0.67 для “middle”)
- MiniLM + RandomForest/LightGBM/CatBoost с oversampling (accuracy ~0.50, f1 по двум основным классам > 0.45)

Попытки дообучить Roberta или использовать SMOTE не дали прироста качества из-за малого объёма данных и дисбаланса классов.

**Главный практический вывод:**  
Для реальных бизнес-задач “умные” эмбеддинги (MiniLM) + ансамбли или логистическая регрессия дают надёжный результат на ограниченных и дисбалансированных данных, если применить балансировку классов.

---

Дальнейшее улучшение качества возможно только при увеличении объёма данных или применении более продвинутых методов feature engineering.
